In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from scipy.spatial import distance

from tqdm.auto import tqdm
import pickle
import gc
import json
import pandas as pd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# == recnn ==
import sys
sys.path.append("../../")
import recnn


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1t0LNCbqLjiLkAMFwtP8OIYU-zPUCNAjK
meta = json.load(open('../../data/parsed/omdb.json'))
tqdm.pandas()

In [3]:
frame_size = 10
batch_size = 1
# embeddgings: https://drive.google.com/open?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
dirs = recnn.data.env.DataPath(
    base="../../data/",
    embeddings="embeddings/ml20_pca128.pkl",
    ratings="ml-20m/ratings.csv",
    cache="cache/frame_env.pkl", # cache will generate after you run
    use_cache=True
)
env = recnn.data.env.FrameEnv(dirs, frame_size, batch_size)

In [4]:
ddpg = recnn.nn.models.Actor(1290, 128, 256).to(cuda)
td3 = recnn.nn.models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../../models/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../../models/td3_policy.pt'))

<All keys matched successfully>

In [5]:
test_batch = next(iter(env.test_dataloader))
state, action, reward, next_state, done = recnn.data.get_base_batch(test_batch)

In [15]:
def rank(gen_action, metric):
    scores = []
    for i in env.base.key_to_id.keys():
        if i == 0 or i == '0':
            continue
        scores.append([i, metric(env.base.embeddings[env.base.key_to_id[i]], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])]['omdb'][key]  for key in ['Title',
                                'Genre', 'Language', 'Released', 'imdbRating']])
        # scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'Title', 'Genre', 'Language', 'Released', 'imdbRating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [16]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [17]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,33974,33.136658,For Pete's Sake,Comedy,English,13 Sep 1974,6.6
1,155,33.171162,Beyond Rangoon,"Action, Adventure, Drama",English,25 Aug 1995,6.6
2,73286,33.221085,Diamond Dogs,"Action, Adventure","English, Mandarin",01 Nov 2009,4.2
3,84189,33.505642,I Spit on Your Grave,"Horror, Thriller",English,20 Jan 2011,6.3
4,6378,33.833809,The Italian Job,"Action, Crime, Thriller","English, Russian, Italian",30 May 2003,7.0
5,3124,33.860462,Agnes Browne,"Comedy, Drama, Romance","English, French",03 Mar 2000,6.6
6,183,33.861977,Mute Witness,"Crime, Thriller","English, Russian",15 Sep 1995,6.7
7,99901,33.895218,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5
8,101483,33.952557,Simpaticul domn R,"Action, Crime",Romanian,19 Jan 1970,5.3
9,8741,33.972130,Ruby Cairo,"Mystery, Thriller","English, Spanish, Arabic",29 Oct 1993,5.0


In [18]:
rank(ddpg_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,73286,0.697688,Diamond Dogs,"Action, Adventure","English, Mandarin",01 Nov 2009,4.2
1,115991,0.709012,"Kumiko, the Treasure Hunter",Drama,"English, Japanese",20 Feb 2015,6.6
2,33974,0.709176,For Pete's Sake,Comedy,English,13 Sep 1974,6.6
3,155,0.710086,Beyond Rangoon,"Action, Adventure, Drama",English,25 Aug 1995,6.6
4,183,0.710300,Mute Witness,"Crime, Thriller","English, Russian",15 Sep 1995,6.7
5,6378,0.714380,The Italian Job,"Action, Crime, Thriller","English, Russian, Italian",30 May 2003,7.0
6,236,0.715951,French Kiss,"Comedy, Drama, Romance","English, French",05 May 1995,6.6
7,8665,0.717257,The Bourne Supremacy,"Action, Mystery, Thriller","English, Russian, German, Italian",23 Jul 2004,7.7
8,8711,0.720883,Mr. Blandings Builds His Dream House,"Comedy, Romance",English,04 Jun 1948,7.3
9,115624,0.723578,Dream Home,"Crime, Drama, Horror","Cantonese, Mandarin, Tagalog, English",13 May 2010,6.7


In [19]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,Title,Genre,Language,Released,imdbRating
0,51847,0.704798,Right at Your Door,"Drama, Sci-Fi, Thriller",English,08 Sep 2006,6.1
1,155,0.705639,Beyond Rangoon,"Action, Adventure, Drama",English,25 Aug 1995,6.6
2,73286,0.712891,Diamond Dogs,"Action, Adventure","English, Mandarin",01 Nov 2009,4.2
3,8665,0.713895,The Bourne Supremacy,"Action, Mystery, Thriller","English, Russian, German, Italian",23 Jul 2004,7.7
4,33974,0.714483,For Pete's Sake,Comedy,English,13 Sep 1974,6.6
5,183,0.715246,Mute Witness,"Crime, Thriller","English, Russian",15 Sep 1995,6.7
6,115991,0.718142,"Kumiko, the Treasure Hunter",Drama,"English, Japanese",20 Feb 2015,6.6
7,115624,0.723090,Dream Home,"Crime, Drama, Horror","Cantonese, Mandarin, Tagalog, English",13 May 2010,6.7
8,78420,0.724629,The House on Telegraph Hill,"Crime, Drama, Film-Noir, Mystery, Thriller",English,27 Aug 1951,7.0
9,84189,0.724824,I Spit on Your Grave,"Horror, Thriller",English,20 Jan 2011,6.3


In [20]:
rank(ddpg_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,81364,80.710885,The Box,"Crime, Thriller",English,27 Sep 2007,4.8
1,5661,81.250414,Shamus,"Action, Comedy, Crime",English,05 Feb 1973,6.0
2,70867,81.440351,Lonely Street,"Comedy, Mystery, Thriller",English,14 Sep 2008,5.2
3,8834,81.683950,The Cookout,Comedy,English,03 Sep 2004,3.7
4,26183,81.815733,Asterix and Cleopatra,"Animation, Adventure, Comedy, Family, History",French,24 Sep 1969,7.2
5,114510,81.907056,Gang Tapes,"Crime, Drama",English,01 Feb 2001,6.1
6,27376,81.938139,The Tunnel,"Drama, History, Thriller",German,21 Jan 2001,7.8
7,31152,82.255873,For Love and Gold,"Adventure, Comedy, History","Italian, Latin",27 Sep 1983,7.9
8,5626,82.296118,Shanghai Ghetto,Documentary,"English, German, Mandarin",28 Feb 2002,7.1
9,129066,82.445785,Our Curse,"Documentary, Short, Drama","Polish, Polish",17 Oct 2014,7.2


In [21]:
rank(ddpg_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,33974,33.136658,For Pete's Sake,Comedy,English,13 Sep 1974,6.6
1,155,33.171162,Beyond Rangoon,"Action, Adventure, Drama",English,25 Aug 1995,6.6
2,73286,33.221085,Diamond Dogs,"Action, Adventure","English, Mandarin",01 Nov 2009,4.2
3,84189,33.505642,I Spit on Your Grave,"Horror, Thriller",English,20 Jan 2011,6.3
4,6378,33.833809,The Italian Job,"Action, Crime, Thriller","English, Russian, Italian",30 May 2003,7.0
5,3124,33.860462,Agnes Browne,"Comedy, Drama, Romance","English, French",03 Mar 2000,6.6
6,183,33.861977,Mute Witness,"Crime, Thriller","English, Russian",15 Sep 1995,6.7
7,99901,33.895218,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5
8,101483,33.952557,Simpaticul domn R,"Action, Crime",Romanian,19 Jan 1970,5.3
9,8741,33.972130,Ruby Cairo,"Mystery, Thriller","English, Spanish, Arabic",29 Oct 1993,5.0


In [22]:
rank(ddpg_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,66059,6.827259,Bad Luck Love,"Crime, Drama",Finnish,29 Sep 2000,6.0
1,118278,6.843920,In Your Hands,Drama,French,02 Feb 2011,5.5
2,40148,6.879690,Revolver,"Action, Crime, Drama, Mystery, Thriller","English, Cantonese",22 Sep 2005,6.4
3,55080,6.892529,The Brave One,"Action, Crime, Drama, Thriller",English,14 Sep 2007,6.8
4,3858,6.899030,Cecil B. DeMented,"Comedy, Crime, Thriller",English,01 Sep 2000,6.2
5,5822,6.905793,Acción mutante,"Comedy, Sci-Fi",Spanish,03 Feb 1993,6.5
6,6113,6.907848,Passion,"Comedy, Drama","French, German, Polish",26 May 1982,6.2
7,906,6.908847,Gaslight,"Crime, Drama, Film-Noir, Mystery, Thriller","English, Italian",30 Oct 1944,7.8
8,93928,6.958758,After...,"Horror, Thriller",English,04 Nov 2006,3.2
9,102969,6.999890,The Era of Vampires,"Horror, Action",Cantonese,23 May 2003,4.7


In [23]:
rank(ddpg_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,115624,0.744867,Dream Home,"Crime, Drama, Horror","Cantonese, Mandarin, Tagalog, English",13 May 2010,6.7
1,27020,0.753229,Gia,"Biography, Drama, Romance","English, German, French",31 Jan 1998,7.0
2,127484,0.758389,Karen Cries on the Bus,Drama,Spanish,13 May 2011,6.7
3,5492,0.760725,Fathom,"Adventure, Comedy",English,09 Aug 1967,5.7
4,8711,0.762831,Mr. Blandings Builds His Dream House,"Comedy, Romance",English,04 Jun 1948,7.3
5,8665,0.765452,The Bourne Supremacy,"Action, Mystery, Thriller","English, Russian, German, Italian",23 Jul 2004,7.7
6,2250,0.766881,Men Don't Leave,Drama,English,23 Feb 1990,6.7
7,8645,0.769787,Maria Full of Grace,"Crime, Drama","Spanish, English",06 Aug 2004,7.5
8,33974,0.771442,For Pete's Sake,Comedy,English,13 Sep 1974,6.6
9,81180,0.774892,30 Days of Night: Dark Days,"Horror, Thriller",English,05 Oct 2010,4.4


In [24]:
rank(ddpg_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,5492,297.301147,Fathom,"Adventure, Comedy",English,09 Aug 1967,5.7
1,33974,297.394287,For Pete's Sake,Comedy,English,13 Sep 1974,6.6
2,7616,300.544159,Body Double,"Crime, Drama, Mystery, Thriller",English,26 Oct 1984,6.8
3,3124,301.977234,Agnes Browne,"Comedy, Drama, Romance","English, French",03 Mar 2000,6.6
4,81398,302.326874,Eastern Drift,Drama,"French, Lithuanian, Russian, Polish",08 Dec 2010,6.5
5,73286,302.747070,Diamond Dogs,"Action, Adventure","English, Mandarin",01 Nov 2009,4.2
6,155,302.858002,Beyond Rangoon,"Action, Adventure, Drama",English,25 Aug 1995,6.6
7,84189,303.024323,I Spit on Your Grave,"Horror, Thriller",English,20 Jan 2011,6.3
8,105,304.168213,The Bridges of Madison County,"Drama, Romance",English,02 Jun 1995,7.6
9,81180,304.428772,30 Days of Night: Dark Days,"Horror, Thriller",English,05 Oct 2010,4.4


# TD3

In [25]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [26]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,103498,17.341663,Double Solitaire,Drama,English,16 Jan 1974,N/A
1,1316,17.439983,Anna Meintjies,Drama,English,13 Nov 1996,N/A
2,95230,17.456381,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
3,108495,17.723240,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
4,73639,17.790436,"Artista, dolarii si Ardelenii",Western,"English, Romanian",22 Dec 1978,7.3
5,112844,17.855919,Bus Stop,Comedy,English,22 Aug 1982,6.9
6,80768,17.896160,Patsy,"Comedy, Drama, Romance",English,20 Jun 2008,7.6
7,73987,17.914110,"Pruncul, petrolul si Ardelenii","Comedy, Western",English,12 Jan 1981,7.6
8,107836,18.068710,Annie Get Your Gun,Musical,English,19 Mar 1967,7.2
9,93885,18.166391,Osmosis,"Comedy, Romance, Drama","French, English",05 May 2004,6.0


In [27]:
rank(td3_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,116323,0.611751,Cry of the City,"Crime, Drama, Film-Noir","English, Italian",11 Oct 1948,7.3
1,1659,0.645251,Hurricane Streets,"Crime, Drama, Romance",English,19 Feb 1998,6.5
2,5088,0.653687,Going Places,"Action, Comedy, Crime, Drama",French,13 May 1974,7.3
3,1740,0.658109,Men of Means,"Action, Drama",English,25 May 1999,6.1
4,1910,0.668964,I Went Down,"Action, Comedy, Crime, Drama",English,24 Jun 1998,7.0
5,80677,0.671105,Classe Tous Risques,"Crime, Drama, Romance, Thriller","French, Italian",01 Apr 1960,7.5
6,4801,0.671588,The Little Foxes,"Drama, Romance",English,29 Aug 1941,8.2
7,66618,0.673443,A World Without Thieves,"Action, Crime, Drama",Mandarin,09 Dec 2004,7.3
8,40291,0.677181,Siblings,Comedy,English,20 Sep 2005,6.6
9,121152,0.678059,The White Haired Witch of Lunar Kingdom,Fantasy,Mandarin,25 Apr 2014,5.3


In [28]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,Title,Genre,Language,Released,imdbRating
0,116323,0.612806,Cry of the City,"Crime, Drama, Film-Noir","English, Italian",11 Oct 1948,7.3
1,5088,0.619887,Going Places,"Action, Comedy, Crime, Drama",French,13 May 1974,7.3
2,85892,0.643079,Wild Rovers,Western,English,13 Oct 1971,6.6
3,1659,0.643753,Hurricane Streets,"Crime, Drama, Romance",English,19 Feb 1998,6.5
4,1740,0.649078,Men of Means,"Action, Drama",English,25 May 1999,6.1
5,66618,0.651722,A World Without Thieves,"Action, Crime, Drama",Mandarin,09 Dec 2004,7.3
6,40291,0.658819,Siblings,Comedy,English,20 Sep 2005,6.6
7,1910,0.659891,I Went Down,"Action, Comedy, Crime, Drama",English,24 Jun 1998,7.0
8,110627,0.661642,Hit Lady,"Action, Drama, Thriller",English,08 Oct 1974,5.8
9,4801,0.668348,The Little Foxes,"Drama, Romance",English,29 Aug 1941,8.2


In [29]:
rank(td3_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,101360,77.215319,The Call,"Crime, Drama, Horror, Thriller",English,15 Mar 2013,6.7
1,83796,77.408417,Anything for Her,"Crime, Drama, Thriller","French, Spanish",03 Dec 2008,7.1
2,112023,78.014115,Annabel Takes a Tour,Comedy,English,11 Nov 1938,5.7
3,121594,78.253820,Midnight Mary,"Crime, Drama, Romance",English,30 Jun 1933,7.1
4,87890,78.297705,The Carey Treatment,"Crime, Mystery, Thriller",English,02 Jul 1972,6.2
5,84158,78.872487,Craig's Wife,Drama,English,25 Sep 1936,7.4
6,89377,79.036825,Wrecked,"Adventure, Drama, Mystery, Thriller",English,10 Jun 2011,5.3
7,77944,79.160625,Art & Copy,Documentary,English,21 Aug 2009,6.8
8,101294,79.270645,Shanghai Calling,"Comedy, Drama, Romance","English, Mandarin, Shanghainese",08 Feb 2013,6.3
9,3770,79.395836,Dreamscape,"Action, Adventure, Horror, Sci-Fi, Thriller",English,17 Aug 1984,6.3


In [30]:
rank(td3_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,103498,17.341663,Double Solitaire,Drama,English,16 Jan 1974,N/A
1,1316,17.439983,Anna Meintjies,Drama,English,13 Nov 1996,N/A
2,95230,17.456381,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
3,108495,17.723240,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
4,73639,17.790436,"Artista, dolarii si Ardelenii",Western,"English, Romanian",22 Dec 1978,7.3
5,112844,17.855919,Bus Stop,Comedy,English,22 Aug 1982,6.9
6,80768,17.896160,Patsy,"Comedy, Drama, Romance",English,20 Jun 2008,7.6
7,73987,17.914110,"Pruncul, petrolul si Ardelenii","Comedy, Western",English,12 Jan 1981,7.6
8,107836,18.068710,Annie Get Your Gun,Musical,English,19 Mar 1967,7.2
9,93885,18.166391,Osmosis,"Comedy, Romance, Drama","French, English",05 May 2004,6.0


In [31]:
rank(td3_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,73639,4.538842,"Artista, dolarii si Ardelenii",Western,"English, Romanian",22 Dec 1978,7.3
1,1316,4.559927,Anna Meintjies,Drama,English,13 Nov 1996,N/A
2,122490,4.628144,Wicked Blood,"Action, Crime, Drama, Thriller",English,04 Mar 2014,5.4
3,103498,4.637841,Double Solitaire,Drama,English,16 Jan 1974,N/A
4,1774,4.648876,Mass Transit,Short,English,12 Feb 1998,6.8
5,95230,4.689195,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
6,108495,4.755808,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
7,73987,4.773306,"Pruncul, petrolul si Ardelenii","Comedy, Western",English,12 Jan 1981,7.6
8,739,4.806682,Honigmond,Comedy,German,11 Apr 1996,4.2
9,131052,4.829328,Die Weihnachtsklempner,Comedy,German,N/A,6.5


In [32]:
rank(td3_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,100737,0.709361,Snitch,"Action, Drama, Thriller","English, Spanish",22 Feb 2013,6.4
1,101360,0.709755,The Call,"Crime, Drama, Horror, Thriller",English,15 Mar 2013,6.7
2,116323,0.709861,Cry of the City,"Crime, Drama, Film-Noir","English, Italian",11 Oct 1948,7.3
3,121594,0.713298,Midnight Mary,"Crime, Drama, Romance",English,30 Jun 1933,7.1
4,112023,0.713565,Annabel Takes a Tour,Comedy,English,11 Nov 1938,5.7
5,100326,0.720144,Stand Up Guys,"Comedy, Crime, Thriller","English, Belarusian",01 Feb 2013,6.5
6,50442,0.724795,Alpha Dog,"Biography, Crime, Drama, Thriller",English,12 Jan 2007,6.9
7,2155,0.730748,Slums of Beverly Hills,"Comedy, Drama",English,11 Sep 1998,6.6
8,101823,0.731611,Poor White Trash,"Comedy, Crime",English,02 May 2000,5.1
9,1659,0.733192,Hurricane Streets,"Crime, Drama, Romance",English,19 Feb 1998,6.5


In [33]:
rank(td3_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,80768,160.847870,Patsy,"Comedy, Drama, Romance",English,20 Jun 2008,7.6
1,103498,161.337524,Double Solitaire,Drama,English,16 Jan 1974,N/A
2,1316,162.711624,Anna Meintjies,Drama,English,13 Nov 1996,N/A
3,95230,163.111053,Macbeth in Manhattan,Drama,English,14 Mar 1999,5.6
4,108495,163.214966,Zincirbozan,"Crime, Drama",Turkish,13 Apr 2007,5.8
5,70611,164.001160,Traffic Affairs,Drama,German,17 Nov 2005,6.6
6,111601,164.060303,Die Fischerin,Drama,German,23 May 2014,6.3
7,93885,164.784149,Osmosis,"Comedy, Romance, Drama","French, English",05 May 2004,6.0
8,87063,165.097488,Bis zum Ellenbogen,Comedy,German,01 Nov 2007,6.4
9,99901,165.244781,The Almighty Tycoon of the Ninja,"Action, Comedy, Sci-Fi",Greek,09 Mar 2007,5.5


# BCQ

In [71]:
perturbator = recnn.models.bcqPerturbator(1290, 128, 256).to(cuda)
generator = recnn.models.bcqGenerator(1290, 128, 512).to(cuda)

perturbator.load_state_dict(torch.load('../../models/bcq_perturbator.pt'))
generator.load_state_dict(torch.load('../../models/bcq_generator.pt'))

<All keys matched successfully>

In [72]:
sampled_actions = generator.decode(state)
perturbed_actions= perturbator(state, sampled_actions)

In [73]:
bcq_action = perturbed_actions[np.random.randint(0,
                               sampled_actions.size(0), 1)[0]].detach().cpu().numpy()

In [74]:
rank(bcq_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,3340,27.769102,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
1,3638,27.866560,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
2,2270,28.175209,A Century of Cinema,Documentary,English,N/A,5.6
3,113128,28.272356,Return to Life,Documentary,English,20 Jul 1938,6.0
4,77734,28.288343,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
5,97304,28.448215,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
6,122458,28.585938,Supernova,"Action, Adventure, Drama","English, French",05 Sep 2005,3.7
7,80160,28.630526,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6
8,5497,28.633619,Our Man Flint,"Action, Adventure, Comedy, Fantasy, Sci-Fi","English, Italian",16 Jan 1966,6.5
9,2683,28.641691,Austin Powers: The Spy Who Shagged Me,"Action, Adventure, Comedy, Crime","English, German",11 Jun 1999,6.6


In [75]:
rank(bcq_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,3638,0.614605,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
1,102156,0.620820,Zombies of the Stratosphere,"Action, Sci-Fi",English,16 Jul 1952,5.7
2,3340,0.627883,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
3,3659,0.632156,Quatermass 2,"Sci-Fi, Horror",English,01 Sep 1957,6.9
4,97304,0.642555,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
5,102090,0.642729,The Magnetic Monster,Sci-Fi,English,18 Feb 1953,5.8
6,2376,0.646069,A View to a Kill,"Action, Adventure, Thriller","English, French",24 May 1985,6.4
7,71254,0.650713,Gamer,"Action, Sci-Fi, Thriller",English,04 Sep 2009,5.8
8,8878,0.650853,Class of Nuke 'Em High Part II: Subhumanoid Me...,"Comedy, Horror, Sci-Fi",English,12 Apr 1991,4.4
9,66808,0.651965,Far Cry,"Action, Adventure, Sci-Fi",English,02 Oct 2008,3.1


In [76]:
rank(bcq_action, distance.correlation)

,id,score,Title,Genre,Language,Released,imdbRating
0,3638,0.614694,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
1,102156,0.626638,Zombies of the Stratosphere,"Action, Sci-Fi",English,16 Jul 1952,5.7
2,3340,0.628047,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
3,3659,0.634837,Quatermass 2,"Sci-Fi, Horror",English,01 Sep 1957,6.9
4,102090,0.640015,The Magnetic Monster,Sci-Fi,English,18 Feb 1953,5.8
5,97304,0.640677,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
6,2376,0.642237,A View to a Kill,"Action, Adventure, Thriller","English, French",24 May 1985,6.4
7,71254,0.646934,Gamer,"Action, Sci-Fi, Thriller",English,04 Sep 2009,5.8
8,122458,0.653172,Supernova,"Action, Adventure, Drama","English, French",05 Sep 2005,3.7
9,1255,0.653200,Bad Taste,"Comedy, Horror, Sci-Fi","English, Maori",01 Jun 1989,6.6


In [77]:
rank(bcq_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,1258,77.604380,The Shining,"Drama, Horror",English,13 Jun 1980,8.4
1,85505,78.561295,X Games 3D: The Movie,"Documentary, Sport",English,21 Aug 2009,6.4
2,5586,78.943230,The Gypsy Moths,"Action, Drama, Romance",English,13 Sep 1969,6.4
3,540,79.097669,Sliver,"Drama, Thriller",English,21 May 1993,5.0
4,4511,79.673246,High Spirits,"Comedy, Fantasy",English,18 Nov 1988,5.8
5,3174,80.188363,Man on the Moon,"Biography, Comedy, Drama",English,22 Dec 1999,7.4
6,85268,80.409850,Scooby-Doo! The Mystery Begins,"Animation, Adventure, Comedy, Family, Fantasy,...",English,13 Sep 2009,5.4
7,524,80.586946,Rudy,"Biography, Drama, Sport","English, Spanish",22 Oct 1993,7.5
8,111726,80.622281,Scooby-Doo! WrestleMania Mystery,"Animation, Comedy, Family, Mystery",English,25 Mar 2014,6.3
9,89112,80.775261,The Kid from Brooklyn,"Comedy, Musical",English,21 Mar 1946,6.7


In [78]:
rank(bcq_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,3340,27.769102,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
1,3638,27.866560,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
2,2270,28.175209,A Century of Cinema,Documentary,English,N/A,5.6
3,113128,28.272356,Return to Life,Documentary,English,20 Jul 1938,6.0
4,77734,28.288343,Confessions of a Gangsta,Action,English,26 Sep 2006,4.3
5,97304,28.448215,Argo,"Biography, Drama, Thriller","English, Persian, Arabic",12 Oct 2012,7.7
6,122458,28.585938,Supernova,"Action, Adventure, Drama","English, French",05 Sep 2005,3.7
7,80160,28.630526,The Flintstone Kids' Just Say No Special,Animation,English,15 Sep 1988,5.6
8,5497,28.633619,Our Man Flint,"Action, Adventure, Comedy, Fantasy, Sci-Fi","English, Italian",16 Jan 1966,6.5
9,2683,28.641691,Austin Powers: The Spy Who Shagged Me,"Action, Adventure, Comedy, Crime","English, German",11 Jun 1999,6.6


In [79]:
rank(bcq_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,71254,5.826878,Gamer,"Action, Sci-Fi, Thriller",English,04 Sep 2009,5.8
1,4678,5.872302,UHF,"Comedy, Drama",English,21 Jul 1989,7.0
2,57692,5.921563,That Man from Rio,"Action, Adventure, Comedy, Romance","French, English, Portuguese",05 Feb 1964,7.1
3,45106,5.924949,American Dreamz,"Comedy, Music","English, Mandarin",21 Apr 2006,5.5
4,66152,5.952034,TerrorVision,"Comedy, Horror, Sci-Fi",English,14 Feb 1986,5.7
5,103460,5.971830,Ghoulies Go to College,"Comedy, Fantasy, Horror",English,18 Sep 1991,3.9
6,81132,5.972182,Rubber,"Comedy, Fantasy, Horror",English,10 Nov 2010,5.8
7,4009,5.990061,Talk Radio,Drama,English,13 Jan 1989,7.3
8,6482,5.996605,Dumb and Dumberer: When Harry Met Lloyd,Comedy,English,13 Jun 2003,3.4
9,79599,6.007491,Operation: Endgame,"Action, Comedy",English,19 May 2012,4.9


In [80]:
rank(bcq_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,8878,0.704326,Class of Nuke 'Em High Part II: Subhumanoid Me...,"Comedy, Horror, Sci-Fi",English,12 Apr 1991,4.4
1,3638,0.712082,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
2,3340,0.717532,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
3,5586,0.717617,The Gypsy Moths,"Action, Drama, Romance",English,13 Sep 1969,6.4
4,31950,0.720379,Curse of the Demon,"Fantasy, Horror, Mystery, Thriller",English,01 Jul 1958,7.5
5,66808,0.720789,Far Cry,"Action, Adventure, Sci-Fi",English,02 Oct 2008,3.1
6,110380,0.724224,Nitro Circus: The Movie,"Documentary, Action, Comedy",English,08 Aug 2012,6.7
7,69626,0.724730,Houdini,"Biography, Drama",English,02 Jul 1953,6.9
8,7423,0.725031,Hitman Hart: Wrestling with Shadows,"Documentary, Biography, Sport",English,20 Dec 1998,8.2
9,94122,0.725402,Comic-Con Episode IV: A Fan's Hope,Documentary,English,23 Nov 2012,7.0


In [81]:
rank(bcq_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,31950,240.192123,Curse of the Demon,"Fantasy, Horror, Mystery, Thriller",English,01 Jul 1958,7.5
1,1258,249.180908,The Shining,"Drama, Horror",English,13 Jun 1980,8.4
2,110792,250.279404,Ghost Adventures,Documentary,English,N/A,8.0
3,3638,251.641708,Moonraker,"Action, Adventure, Sci-Fi, Thriller","English, Italian, Portuguese",29 Jun 1979,6.3
4,26429,253.122864,Love at First Bite,Comedy,English,27 Apr 1979,6.1
5,3144,253.795853,The Glass Bottom Boat,"Comedy, Romance",English,09 Jun 1966,6.5
6,3340,254.168716,Bride of the Monster,"Horror, Sci-Fi",English,01 Feb 1956,4.2
7,2793,254.434235,An American Werewolf in Paris,"Comedy, Fantasy, Horror, Thriller","English, French, German",25 Dec 1997,5.1
8,2455,254.552689,The Fly,"Drama, Horror, Sci-Fi",English,15 Aug 1986,7.5
9,104835,254.683899,The Incredible Melting Man,"Horror, Sci-Fi",English,01 Dec 1977,3.9
